In [20]:
import pandas as pd
csv_market = pd.read_csv('G:/공유 드라이브/통분방+공모전/데이터/전국_전통시장_표준데이터_20220706.csv',
                        encoding='cp949')

data = {'위도' : csv_market['위도'], '경도' : csv_market['경도']}
df = pd.DataFrame(data)

In [32]:
import requests
import json

lst_conv = []
lst_mart = []
lst_subw = []
lst_cafe = []
lst_food = []
lst_scho = []

for row in df.iterrows():
    url = "https://dapi.kakao.com/v2/local/search/keyword.json?y="+str(row[1]['위도'])+"&x="+str(row[1]['경도'])+"&radius=1000"
    
    query_conv = {'query' : '편의점'}
    query_mart = {'query' : '대형마트'}
    query_subw = {'query' : '지하철역'}
    query_cafe = {'query' : '카페'}
    query_food = {'query' : '음식점'}
    query_scho = {'query' : '학교'}
    
    header = {'Authorization': 'KakaoAK a2fba27f3df9c58f334f05f14025e5c7'}
    
    response_conv = requests.get(url, headers=header, params=query_conv)
    response_mart = requests.get(url, headers=header, params=query_mart)
    response_subw = requests.get(url, headers=header, params=query_subw)
    response_cafe = requests.get(url, headers=header, params=query_cafe)
    response_food = requests.get(url, headers=header, params=query_food)
    response_scho = requests.get(url, headers=header, params=query_scho)
    
    tokens_conv = response_conv.json()
    tokens_mart = response_mart.json()
    tokens_subw = response_subw.json()
    tokens_cafe = response_cafe.json()
    tokens_food = response_food.json()
    tokens_scho = response_scho.json()
    
    lst_conv.append(tokens_conv['meta']['total_count'])
    
    if tokens_mart['meta']['total_count'] == 0:
        lst_mart.append(1)
    else:
        lst_mart.append(0)
        
    if tokens_subw['meta']['total_count'] == 0:
        lst_subw.append(0)
    else:
        lst_subw.append(1)
        
    lst_cafe.append(tokens_cafe['meta']['total_count'])
    lst_food.append(tokens_food['meta']['total_count'])
    
    num = 0
    for i in range(0, min(tokens_scho['meta']['total_count'],14)):
        split = tokens_scho['documents'][i]['category_name'].split('>')
        if len(split)>1 and split[1] == ' 학교 ':
            num += 1
    lst_scho.append(num)

In [33]:
people = pd.read_excel('G:/공유 드라이브/통분방+공모전/데이터/ㄱㄱ전통시장현황_최종 사본.xlsx')['유동인구'].tolist()
df_market = pd.DataFrame({'편의점 수' : lst_conv,
                         '대형마트 여부' : lst_mart,
                         '지하철역 여부' : lst_subw,
                         '카페 수' : lst_cafe,
                         '음식점 수' : lst_food,
                         '학교 수' : lst_scho,
                         '유동인구' : people},
                        index = csv_market['시장명'])
df_market.to_csv('G:/공유 드라이브/통분방+공모전/데이터/카카오데이터.csv', encoding='euc-kr')